In [1]:
import os
import sys
import re
#import MeCab
from bs4 import BeautifulSoup
import kss
import json
import numpy as np 
import pandas as pd
from tqdm import tqdm
import argparse
import pickle

PROBLEM = 'ext'

## 사용할 path 정의
# PROJECT_DIR = '/home/uoneway/Project/PreSumm_ko'
PROJECT_DIR = '/home/ubuntu/KoBertSum'

DATA_DIR = f'{PROJECT_DIR}/{PROBLEM}/data'
RAW_DATA_DIR1 = DATA_DIR + '/kmooc_test'
RAW_DATA_DIR2 = DATA_DIR + '/raw'
JSON_DATA_DIR = DATA_DIR + '/json_data'
BERT_DATA_DIR = DATA_DIR + '/bert_data' 
LOG_DIR = f'{PROJECT_DIR}/{PROBLEM}/logs'
LOG_PREPO_FILE = LOG_DIR + '/preprocessing.log' 

[Korean Sentence Splitter]: Initializing Kss...


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("-task", default=None, type=str, choices=['df', 'train_bert', 'test_bert'])
parser.add_argument("-target_summary_sent", default='abs', type=str)
parser.add_argument("-n_cpus", default='2', type=str)
parser.add_argument("-file", default=None, type=str)

_StoreAction(option_strings=['-file'], dest='file', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help=None, metavar=None)

In [3]:
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RAW_DATA_DIR1, exist_ok=True)

In [4]:
import re
l=list()
i=0
file=None
file_list = os.listdir(RAW_DATA_DIR1)
if file==None:
    for now in file_list:
        if not 'txt' in now:
            continue
        with open (f'{RAW_DATA_DIR1}/{now}', 'r', encoding='utf-8') as f:
            i=i+1
            get = re.split('\.|\?|\!|\/', f.read())
            l.append({'id':i, 'article_original':get})
else:
    with open (f'{RAW_DATA_DIR1}/{now}', 'r', encoding='utf-8') as f:
        i=i+1
        get = re.split('\.|\?|\!|\/', f.read())
        l.append({'id':i, 'article_original':get})


In [5]:
df = pd.DataFrame(l)
df

,id,article_original
0,1,"[안녕하십니까, 여러분들 모두가 지구 과학의 신이 되는 그날까지, 지구과학 가르..."
1,2,"[우리 지난 시간에 대륙 이동설 베게너가 주장했던 거죠, 대륙 이동설에 대해서 학..."
2,3,"[그리고 이렇게 대륙 이동설과 맨틀 대류설 그리고 해저 확장설에 이르기까지, \n\..."
3,4,[그러면 다르게 말하면 어떻게 말할 수가 있냐 하면 지구상에서 일어나는 지진이나 화...
4,5,[그럼 이제 본격적으로 판 구조론에 대해서 알아보기 전에 이 판의 정의에 대해서 살...
5,6,"[그래서 만났을 때 야, 판이 뭔데라고 물어보면 이 지각과 이 지각과 판의 개념을..."
6,7,"[그럼 이 상태에서 본격적으로 판에 대한 이야기를 한번 해보도록 할 텐데요, 왼쪽..."
7,8,[그래서 연약권이 요런 부분이다라고 생각해 주시면 되겠는데 우리가 이 연약권을 대략...
8,9,"[그러면 연약권 같은 경우에는 말랑말랑해, 그래서 움직일 수 있어, 이렇게 말씀..."
9,10,[그림을 통해서 그거 파악해 주시면 되겠고 또 한 가지만 더 한 가지만 더 말씀을 ...


In [6]:
df.to_pickle(f'{RAW_DATA_DIR1}/ebs_pickle')
print(f'df({len(df)}) is exported')

df(13) is exported


In [7]:
def preprocessing(text, tokenizer=None):
    text = noise_remove(text)
    if tokenizer is not None:
        text = tokenizer(text)
        text = ' '.join(text)

    return text

In [8]:
def korean_sent_spliter(doc):
    sents_splited = kss.split_sentences(doc)
    if len(sents_splited) == 1:
        # .이나 ?가 있는데도 kss가 분리하지 않은 문장들을 혹시나해서 살펴보니
        # 대부분 쉼표나 가운데점 대신 .을 사용하거나 "" 사이 인용문구 안에 들어가있는 점들. -> 괜찮.
        # aa = sents_splited[0].split('. ')
        # if len(aa) > 1:
        #     print(sents_splited)
        return sents_splited
    else:  # kss로 분리가 된 경우(3문장 이상일 때도 고려)
        #print(sents_splited)
        for i in range(len(sents_splited) - 1):
            idx = 0
            # 두 문장 사이에 .이나 ?가 없는 경우: 그냥 붙여주기
            if sents_splited[idx][-1] not in ['.','?' ] and idx < len(sents_splited) - 1:
                sents_splited[idx] = sents_splited[idx] + ' ' + sents_splited[idx + 1] if doc[len(sents_splited[0])] == ' ' \
                                        else sents_splited[idx] + sents_splited[idx + 1] 
                del sents_splited[idx + 1]
                idx -= 1
        #print(sents_splited)
        return sents_splited

In [9]:
def create_json_files(df, data_type='train', target_summary_sent=None, path=''):
    NUM_DOCS_IN_ONE_FILE = 1000
    start_idx_list = list(range(0, len(df), NUM_DOCS_IN_ONE_FILE))

    for start_idx in tqdm(start_idx_list):
        end_idx = start_idx + NUM_DOCS_IN_ONE_FILE
        if end_idx > len(df):
            end_idx = len(df)  # -1로 하니 안됨...

        #정렬을 위해 앞에 0 채워주기
        length = len(str(len(df)))
        start_idx_str = (length - len(str(start_idx)))*'0' + str(start_idx)
        end_idx_str = (length - len(str(end_idx-1)))*'0' + str(end_idx-1)

        file_name = os.path.join(f'{path}/{data_type}_{target_summary_sent}' \
                                + f'/{data_type}.{start_idx_str}_{end_idx_str}.json') if target_summary_sent is not None \
                    else os.path.join(f'{path}/{data_type}' \
                                + f'/{data_type}.{start_idx_str}_{end_idx_str}.json')
        
        json_list = []
        for i, row in df.iloc[start_idx:end_idx].iterrows():
            original_sents_list = [preprocessing(original_sent).split()  # , korean_tokenizer
                                    for original_sent in row['article_original']]

            summary_sents_list = []
            if target_summary_sent is not None:
                if target_summary_sent == 'ext':
                    summary_sents = row['extractive_sents']
                elif target_summary_sent == 'abs':
                    summary_sents = korean_sent_spliter(row['abstractive'])   
                summary_sents_list = [preprocessing(original_sent).split() # , korean_tokenizer
                                        for original_sent in summary_sents]

            json_list.append({'src': original_sents_list,
                              'tgt': summary_sents_list
            })
            #print(json_list)
        #     break
        # break
        json_string = json.dumps(json_list, indent=4, ensure_ascii=False)
        #print(json_string)
        with open(file_name, 'w+') as json_file:
            json_file.write(json_string)


In [10]:
def noise_remove(text):
    text = text.lower()
    
    # url 대체
    # url_pattern = re.compile(r'https?://\S*|www\.\S*')
    # text = url_pattern.sub(r'URL', text)

    # html 삭제
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")

    # 숫자 중간에 공백 삽입하기
    # text = number_split(text)
    #number_pattern = re.compile('\w*\d\w*') 
#     number_pattern = re.compile('\d+') 
#     text = number_pattern.sub(r'[[NUMBER]]', text)
    

    # PUCTUACTION_TO_REMOVED = string.punctuation.translate(str.maketrans('', '', '\"\'#$%&\\@'))  # !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ 중 적은것을 제외한 나머지를 삭제
    # text = text.translate(str.maketrans(PUCTUACTION_TO_REMOVED, ' '*len(PUCTUACTION_TO_REMOVED))) 

    # remove_redundant_white_spaces
    text = re.sub(' +', ' ', text)

    # tgt special token 으로 활용할 204; 314[ 315] 대체/삭제해줘서 없애주기
    text = re.sub('¶', ' ', text)
    text = re.sub('----------------', ' ', text)
    text = re.sub(';', '.', text)

    return text


In [76]:
bert_data_dir = f"{BERT_DATA_DIR}/ebs_test_bert"
if os.path.exists(bert_data_dir):
    os.system(f"rm {bert_data_dir}/*")
else:
    os.mkdir(bert_data_dir)

json_data_dir = f"{JSON_DATA_DIR}/ebs_test_script"
if os.path.exists(json_data_dir):
        os.system(f"rm {json_data_dir}/*")
else:
    os.mkdir(json_data_dir)
    
create_json_files(df, data_type='ebs_test_script', path=JSON_DATA_DIR)


100%|██████████| 1/1 [00:00<00:00, 42.15it/s]


In [77]:
os.system(f"python preprocess.py"
    + f" -mode format_to_bert -dataset ebs_test_script"
    + f" -raw_path {json_data_dir}"
    + f" -save_path {bert_data_dir}"
    + f" -log_file {LOG_PREPO_FILE}"
    + f" -lower -n_cpus 4")

0

In [78]:
test_file=f'{PROJECT_DIR}/ext/test_ebs/result_ebs_test_step_12000.candidate'
if os.path.exists(test_file):
    os.system(f'rm {test_file}')
    
print(f'result_path={PROJECT_DIR}/ext/test_ebs')

result_path=/home/ubuntu/KoBertSum/ext/test_ebs


In [79]:
os.system(f"python train.py -task ext -mode test \
            -test_from ../ext/models/0909_0929/model_step_12000.pt \
            -bert_data_path {BERT_DATA_DIR}/ebs_test_bert \
            -result_path {PROJECT_DIR}/ext/test_ebs \
            -log_file {PROJECT_DIR}/ext/logs/test_ebs.log \
            -test_batch_size 1  -batch_size 3000 \
            -sep_optim true -use_interval true -visible_gpus 0,1 \
            -max_pos 512 -max_length 200 -alpha 0.95 -min_length 50 \
            -report_rouge False \
            -max_tgt_len 100")

0